In [ ]:
import panel as pn
from ipyleaflet import Map, basemaps, DrawControl, AwesomeIcon, Marker, GeoData, GeoJSON, LayersControl, Popup, WidgetControl, FullScreenControl, CircleMarker
from ipywidgets import Layout, HTML
from datetime import date
import os
import pandas as pd
from io import StringIO
import hvplot.pandas
from bokeh.sampledata.degrees import data as degrees
import geopandas, json
from bokeh.sampledata.sample_geojson import geojson as nhs_feature_collection
pn.extension("ipywidgets", sizing_mode="stretch_width")

# Specify index of the starting/active tab that displays on load.
tabs = pn.Tabs(active=5, tabs_location="left")

In [2]:
## Create widgets.
headings = pn.pane.Markdown('''
## Approximately how many seconds have you lived?
### Enter your information here and see!
''')
name = pn.widgets.TextInput(name="Name", value="Venus")
age = pn.widgets.IntSlider(name="Age", value=50, start=1, end=100)
color = pn.widgets.ColorPicker(name="Color", value="#4f4fdf")
calc_button = pn.widgets.Button(name="Calculate", button_type="primary")
result = pn.widgets.StaticText()

## Create functions.
def compute_secs_lived(event):
  secs = age.value * 365 * 24 * 60 * 60
  result.value = f"Hello {name.value}, you have lived for approximately {secs} seconds!"
  result.style = {"color": color.value}
  return result

## Create interactions between widgets and functions.
calc_button.on_click(compute_secs_lived)

## Create person component and add it as a tab.
person_inputs = pn.Row(name, age, color)
person_component = pn.Column(headings, person_inputs, calc_button, result)
tabs.append(("Person", person_component))

In [3]:
## Create map widgets.
center = [37, -122]     # center = [vertical distance on y-axis, horizontal distance on x-axis] -> [0,0] is underneath Ghana, Africa in South Atlantic Ocean
map = Map(basemap=basemaps.OpenStreetMap.France, center=center, zoom=10, layout=Layout(height="75vh"))
icon = AwesomeIcon(
  name="heart",
  marker_color="blue",
  icon_color="plum",
  spin=True
)
marker = Marker(icon=icon, location=(center[0] - 0.035, center[1] - 0.015), title="Santa Cruz Beach Boardwalk")
map.add_layer(marker)

display_geo_button = pn.widgets.Button(name="Display Geometry", button_type="success")
clear_drawings_button = pn.widgets.Button(name="Clear Drawings")

geo = ""
drawing_geometry = pn.widgets.StaticText(name="Last Drawing's Geometry", value=geo)
count = 0
num_drawings = pn.widgets.StaticText(name="Number of Drawings", value=count)

## Create and customize drawing controls.
draw_control = DrawControl(polyline={}, polygon={}, circle={}, circlemarker={})
draw_control.rectangle = {
  "shapeOptions": {
    "fillColor": "#4f4fdf",
    "color": "#4f4fdf",
    "fillOpacity": 0.3
  }
}
map.add_control(draw_control)

# Save drawn rectangle and increment number of drawings.
def save_drawing(self, action, geo_json):
  global geo, count
  geo = geo_json["geometry"]
  count += 1
  num_drawings.value = count // 2

# Display geometry of recently drawn rectangle and total number of drawings.
def print_drawing_geometry(event):
  # drawing_geometry.value = draw_control.last_draw["geometry"]
  drawing_geometry.value = geo
  

# Clear all drawings and reset drawing count.
def clear_drawings(event):
  draw_control.clear()
  global geo, count
  geo = ""
  drawing_geometry.value = geo
  count = 0
  num_drawings.value = count

## Create interactions between widgets and functions.
draw_control.on_draw(save_drawing)
display_geo_button.on_click(print_drawing_geometry)
clear_drawings_button.on_click(clear_drawings)

## Create map component and add it as a tab.
map_buttons = pn.Row(display_geo_button, clear_drawings_button)
map_component = pn.Column(map, map_buttons, drawing_geometry, num_drawings)
tabs.append(("Map", map_component))

In [4]:
## Create CoastSeg controls with Panel widgets.
date_heading = pn.pane.Markdown('''
## Pick a date:
''')
start_date_picker = pn.widgets.DatePicker(name="Start Date", value=date(2018,12,1))
end_date_picker = pn.widgets.DatePicker(name="End Date", value=date(2019,3,1))
collection_heading = pn.pane.Markdown('''
## Pick a collection:
C01: LandSat Collection 1 (all images prior to 2022-01-01)\n
C02: LandSat Collection 2 (all images after to 2022-01-01)\n
- LandSat 9 is only available in C02
''')
collection_radio_button_group = pn.widgets.RadioButtonGroup(name="Collection", options=["C01", "C02"], button_type="primary", orientation="vertical")
# collection_radio_box_group = pn.widgets.RadioBoxGroup(name="Collection", options=["C01", "C02"], inline=False)
satellites_heading = pn.pane.Markdown('''
## Pick at least one satellite:
Unselect a satellite by clicking on the option again.
''')
satellites_check_button_group = pn.widgets.CheckButtonGroup(name="Satellites", options=["L5", "L7", "L8", "S2"], value=["L8"], button_type="warning")
# satellites_multi_select = pn.widgets.MultiSelect(name="Satellites", options=["L5", "L7", "L8", "S2"], value=["L8"])

## Create CoastSeg controls component and add it as a tab.
date_pickers = pn.Row(start_date_picker, end_date_picker)
coastseg_component = pn.Column(date_heading, date_pickers, collection_heading, collection_radio_button_group, satellites_heading, satellites_check_button_group)
tabs.append(("CoastSeg", coastseg_component))

In [5]:
## Create data widgets.
data_files = os.listdir("data")
file_select = pn.widgets.Select(name="File", options=data_files, value=data_files[0])

# Binds file_select widget's selected value to selected_filename argument.
@pn.depends(file_select)
def get_selected_file_path(selected_filename):
  return "data/" + selected_filename

selected_file_download = pn.widgets.FileDownload(callback=get_selected_file_path, filename=file_select.value, button_type="success", align="end")

# Link value parameter of source (file_select widget) to filename parameter of target (selected_file_download button).
file_select.link(selected_file_download, value="filename")

data = pd.read_csv("data/A_naip_meta_served.csv")
data = data.set_index("label_image_filename")
years_options = list(data.year.unique())
years_multi_choice = pn.widgets.MultiChoice(name="Years", options=years_options, value=[years_options[0]], solid=False)
doodle_spatial_density_range_slider = pn.widgets.RangeSlider(name="Proportion of Image Annotated", start=data.doodle_spatial_density.min(), end=data.doodle_spatial_density.max(), bar_color="#4f4fdf")

# Filter data based on doodle_spatial_density.
@pn.depends(years_multi_choice, doodle_spatial_density_range_slider)
def get_filtered_data(years, density):
  global data
  filtered_data = data
  [min_density, max_density] = density
  if years_multi_choice.value:
    filtered_data = filtered_data[filtered_data.year.isin(years)]
  return filtered_data[(filtered_data.doodle_spatial_density >= min_density) & (filtered_data.doodle_spatial_density <= max_density)]

# Saves filtered data's DataFrame object as a CSV, and converts it into a file StringIO object.
@pn.depends(years_multi_choice, doodle_spatial_density_range_slider)
def get_filtered_file(ymc, dsdrs):
  dataframe = get_filtered_data(ymc, dsdrs)
  sio = StringIO()
  dataframe.to_csv(sio)
  sio.seek(0)
  return sio

filtered_data_file_download = pn.widgets.FileDownload(callback=get_filtered_file, filename="filtered_data.csv", button_type="primary")

## Create data component and add it as a tab.
download_selected_file = pn.Row(file_select, selected_file_download)
filter_controls = pn.Row(years_multi_choice, doodle_spatial_density_range_slider)
info_text = """
  ### Filtered Data
  The following data is a preview of the content in `filtered_data.csv`:
"""
data_component = pn.Column("## Select a File to Download", download_selected_file, "## Filter Data to Download", filter_controls, filtered_data_file_download, pn.pane.Alert(info_text, alert_type="info"), pn.panel(get_filtered_data))
tabs.append(("Data", data_component))

In [6]:
## Create plot widgets.
preview_degrees_data = degrees.head()
degree_options = list(degrees.columns[1:])
degree_multi_choice = pn.widgets.MultiChoice(name="Degree", options=degree_options, value=[degree_options[2], degree_options[3], degree_options[6]], solid=False)

# Creates line plot for all selected degrees.
@pn.depends(degree_multi_choice)
def plot_degree(degree_vals):
  return degrees.hvplot.line(x="Year", y=degree_vals, value_label="% of Degrees Earned by Women", legend="bottom")

nhs_map = Map(center=[53, -3], zoom=6, layout=Layout(height="75vh"))

# Convert JSON data into a GeoDataFrame, which allows us to view the data in a Panel table.
nhs_geodataframe = geopandas.GeoDataFrame.from_features(json.loads(nhs_feature_collection))
preview_nhs_data = nhs_geodataframe.head()

team_info = HTML()
team_popup = Popup(location=[100, -100], child=team_info, name="Team Information")
team_popup.close_popup()    # Initially hide team info popup when the map loads.
nhs_map.add_layer(team_popup)

# Opens a popup that displays team info when clicking/hovering over its marker.
def display_team_info(feature, **kwargs):
  team_popup.location = list(reversed(feature["geometry"]["coordinates"]))
  team_info.value = """
    <b>Name</b>: {}<br>
    <b>Location</b>: {}, {}<br>
    <b>Phone</b>: {}<br>
    <b>Email</b>: {}<br>
    <b>Website</b>: <a href="{}">{}</a>
  """.format(
    feature["properties"]["OrganisationName"],
    feature["properties"]["City"],
    feature["properties"]["County"],
    feature["properties"]["Phone"],
    feature["properties"]["Email"],
    feature["properties"]["Website"],
    feature["properties"]["Website"]
  )
  team_popup.open_popup()

# Option 1: Convert GeoDataFrame into GeoData, which is a layer that can be displayed on the Map.
# nhs_geodata = GeoData(geo_dataframe=nhs_geodataframe, name="NHS Teams")
# nhs_map.add_layer(nhs_geodata)

# Option 2: Convert JSON data into GeoJSON, which is a layer that can be displayed on the Map.
nhs_geojson = GeoJSON(data=json.loads(nhs_feature_collection), hover_style={"width": "25px", "height": "25px", "borderRadius": "50%", "color": "yellow", "fillOpacity": 0.7}, name="NHS Teams")
nhs_geojson.on_click(display_team_info)
nhs_geojson.on_hover(display_team_info)
nhs_map.add_layer(nhs_geojson)

nhs_map.add_control(LayersControl(position="topright"))

## Create plots component and add it as a tab.
degree_subcomponents = pn.Column("## Bachelor's Degrees Earned by Women", preview_degrees_data, degree_multi_choice, plot_degree)
nhs_subcomponents = pn.Column("## GeoJSON Data for [United Kingdom NHS England Area Teams](https://docs.bokeh.org/en/latest/docs/reference/sampledata.html#module-bokeh.sampledata.sample_geojson)", preview_nhs_data, nhs_map)
plots_component = pn.Column(degree_subcomponents, nhs_subcomponents)
tabs.append(("Plots", plots_component))

In [ ]:
## Create customized/styled widgets.


## Create custom component and add it as a tab.
# custom_component = pn.Column()
# tabs.append(("Custom", custom_component))

In [7]:
## Create Elwha widgets.
# elwha_data_types = ["Topography", "Nearshore Bathymetry", "Surface-Sediment Grain-Size Distributions"]
# data_type_select = pn.widgets.Select(name="Type of Data", options=elwha_data_types, value=elwha_data_types[0])

elwha_map = Map(center=[48.148, -123.553], zoom=15, layout=Layout(height="100vh"))

## Plot collected data.
grain_dataframe = pd.read_csv("data/Elwha/Surface-Sediment Grain-Size Distributions/ew18_july_grainsize.csv")

# Gets the time that the data was collected in local time.
def get_time_collected(data):
  return data["Time (GMT)"] if ("Time (GMT)" in data) else data["Time_GMT"]

for (_, row) in grain_dataframe.iterrows():
  # marker = CircleMarker(location=[row["Latitude"], row["Longitude"]], radius=2, color="blue")
  grain_info = HTML(value="""
    <b>Date & Time Collected</b> {} {}<br>
    <b>Sample Type</b> {}<br>
    <b>Weight</b> {}%<br>
    <b>Gravel</b> {}%<br>
    <b>Sand</b> {}%<br>
    <b>Silt</b> {}%<br>
    <b>Clay</b> {}%<br>
    <b>Mud</b> {}%<br>
  """.format(
    row["Date Collected"],
    get_time_collected(row),
    row["Sample Type"],
    row["Wt. percent in -2.00 phi bin"],
    row["Percent Gravel"],
    row["Percent Sand"],
    row["Percent Silt"],
    row["Percent Clay"],
    row["Percent Mud"]
  ))
  marker = Marker(location=[row["Latitude"], row["Longitude"]], popup=grain_info, icon=AwesomeIcon(name="tint"), visible=False, z_index_offset=-1, name="Grain-Size Markers")
  elwha_map.add_layer(marker)

grain_geodataframe = geopandas.GeoDataFrame(grain_dataframe, geometry=geopandas.points_from_xy(grain_dataframe["Longitude"], grain_dataframe["Latitude"]))
grain_geodata = GeoData(geo_dataframe = grain_geodataframe,
  point_style={"fillColor": "blue", "fillOpacity": 0.5, "radius": 8, "weight": 1, "dashArray": 2, "zIndex": 1},
  hover_style={"fillColor": "red" , "fillOpacity": 0.2},
  name = "Surface-Sediment Grain-Size Distributions"
)
elwha_map.add_layer(grain_geodata)

elwha_map.add_control(FullScreenControl())
elwha_map.add_control(LayersControl(position="topright"))

## Create Elwha component and add it as a tab.
elwha_component = pn.Column("## Elwha River Delta Data", elwha_map)
tabs.append(("Elwha", elwha_component))

In [8]:
## Add components to serve on app.
app = pn.Column("# My First Panel App!", tabs)
app.servable()

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] Tabs(active=2, sizing_mode='stretch_width', tabs_location='left')
        [0] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Row(sizing_mode='stretch_width')
                [0] TextInput(name='Name', sizing_mode='stretch_width', value='Venus')
                [1] IntSlider(end=100, name='Age', sizing_mode='stretch_width', start=1, value=50)
                [2] ColorPicker(name='Color', sizing_mode='stretch_width', value='#4f4fdf')
            [2] Button(button_type='primary', name='Calculate', sizing_mode='stretch_width')
            [3] StaticText(sizing_mode='stretch_width')
        [1] Column(sizing_mode='stretch_width')
            [0] IPyLeaflet(Map)
            [1] Row(sizing_mode='stretch_width')
                [0] Button(button_type='success', name='Display Geometry', sizing_mode='stretch_width')
                [1] Button(name='Clear Drawings', sizing_mode='stretch_width')
            [2] StaticText(name="Last Drawing's Geometry", sizing_mode='stretch_width')
            [3] StaticText(name='Number of Drawings', sizing_mode='stretch_width', value=0)
        [2] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Row(sizing_mode='stretch_width')
                [0] DatePicker(name='Start Date', sizing_mode='stretch_width', value=datetime.date(2018, ...)
                [1] DatePicker(name='End Date', sizing_mode='stretch_width', value=datetime.date(2019, 3, 1))
            [2] Markdown(str, sizing_mode='stretch_width')
            [3] RadioButtonGroup(button_type='primary', name='Collection', options=['C01', 'C02'], orientation='vertical', sizing_mode='stretch_width', value='C01')
            [4] Markdown(str, sizing_mode='stretch_width')
            [5] CheckButtonGroup(button_type='warning', name='Satellites', options=['L5', 'L7', 'L8', 'S2'], sizing_mode='stretch_width', value=['L8'])
        [3] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Row(sizing_mode='stretch_width')
                [0] Select(name='File', options=['A_naip_meta_served.csv',...], sizing_mode='stretch_width', value='A_naip_meta_served.csv')
                [1] FileDownload(align='end', button_type='success', callback=<function get_selected_fil..., filename='A_naip_meta_served.csv', label='Download A_naip_meta_serv..., sizing_mode='stretch_width')
            [2] Markdown(str, sizing_mode='stretch_width')
            [3] Row(sizing_mode='stretch_width')
                [0] MultiChoice(name='Years', options=[2014, 2016, 2011, ...], sizing_mode='stretch_width', solid=False, value=[2014])
                [1] RangeSlider(bar_color='#4f4fdf', end=0.862211018199704, name='Proportion of I..., sizing_mode='stretch_width', start=0.000526322858361, value=(0.000526322858361, ..., value_end=0.862211018199704, value_start=0.000526322858361)
            [4] FileDownload(button_type='primary', callback=<function get_filtered_fil..., filename='filtered_data.csv', label='Download filtered_data.cs..., sizing_mode='stretch_width')
            [5] Alert(str, alert_type='info', css_classes=['alert', 'alert-info'], margin=(0, 0, 25, 0), sizing_mode='stretch_width')
            [6] ParamFunction(function, sizing_mode='stretch_width')
        [4] Column(sizing_mode='stretch_width')
            [0] Column(sizing_mode='stretch_width')
                [0] Markdown(str, sizing_mode='stretch_width')
                [1] DataFrame(DataFrame, sizing_mode='stretch_width')
                [2] MultiChoice(name='Degree', options=['Agriculture', ...], sizing_mode='stretch_width', solid=False, value=['Art and Performance', ...])
                [3] ParamFunction(function, sizing_mode='stretch_width')
            [1] Column(sizing_mode='stretch_width')
                [0] Markdown(str, sizing_mode='stret